In [14]:
############################# Q2 Maximal Cuts #############################
######################### Q2(a) MCMC method #########################
import random
import numpy as np
from collections import Counter
from math import exp
# coin that lands head with prob=a_{ij}
def coin(p):
    flip=random.random()
    if flip < p:
        return 1
    return 0

def legalstate(setX,alledge): 
    for i in range(len(setX)):
        if setX[i]==1:
            connect=alledge[i]
            for j in connect:
                if setX[j]==1:
                    return 0
    return 1


def MH_algo(setX,vedge,vertice_num):
    iters=1000
    # cutsize=0
    maxset=[]
    for t in range(iters):
        cutsize=0
        choosev=np.random.randint(0,vertice_num)-1
        nstate=setX[:]
        nstate[choosev]=1-nstate[choosev]
        if legalstate(nstate,vedge)==1:
            aij=exp(2*(Counter(nstate)[1]-Counter(setX)[1]))
            # print('aij',aij)
            if coin(aij)==1:
                setX=nstate
        for i in range(len(setX)):
            if setX[i]==1:
                for j in vedge[i]:
                    if setX[j]==0:
                        cutsize+=1
    return maxset,cutsize

vertice_num=20
edge_num=30
vedge=[[1, 4, 15], [0, 2, 14], [1, 3, 12], [2, 4, 7], [0, 3, 5], 
        [4, 6, 17], [5, 7, 8],[6, 10, 3], [6, 9, 18], [8, 10, 11], 
        [7, 9, 12], [9, 13, 19], [2, 10, 13], [12, 14, 11], [1, 13, 16], 
        [0, 16, 17], [14, 15, 19], [5, 15, 18], [8, 17, 19], [11, 16, 18]]

allsize=[]
allset=[]

iters=300
for t in range(iters):
    setX,cutsize=MH_algo([0]*vertice_num,vedge,vertice_num)
    allsize.append(cutsize)
print('MCMC method')
print('max cut includes',max(allsize),'edges')

MCMC method
max cut includes 24 edges


In [13]:
######################### Q2(b) Cross-Entropy method #########################
import math
import numpy as np
vertice_num=20
edge_num=30
vedge=[[1, 4, 15], [0, 2, 14], [1, 3, 12], [2, 4, 7], [0, 3, 5], 
        [4, 6, 17], [5, 7, 8],[6, 10, 3], [6, 9, 18], [8, 10, 11], 
        [7, 9, 12], [9, 13, 19], [2, 10, 13], [12, 14, 11], [1, 13, 16], 
        [0, 16, 17], [14, 15, 19], [5, 15, 18], [8, 17, 19], [11, 16, 18]]
alledge=[]
for i in range(len(vedge)):
    for j in vedge[i]:
        tmp=[i,j]
        tmp.sort()
        if tmp not in alledge:
            alledge.append(tmp)
# print(alledge)
# print(len(alledge))
p0=[0.5]*vertice_num
rho=0.1
t=1

iters=300
for t in range(iters):
    sortlists=[]
    alllist=[]
    for i in range(200):
        curlist=[]
        for j in range(len(p0)):
            curlist.append(np.random.binomial(1,p0[j]))
        alllist.append(curlist)

    for l in alllist:
        cutsize=0
        for i in range(len(l)):
            if l[i]==1:
                for j in vedge[i]:
                    if l[j]==0:
                        cutsize+=1
        sortlists.append((cutsize,l))
    
    sortlists.sort()
    sortlists.reverse()
    takeout=int(rho*len(sortlists))
    sortlists=sortlists[:takeout]
    
    for i in range(len(p0)):
        cursum=0
        for l in sortlists:
            cursum+=l[1][i]
        p0[i]=cursum/20
# print(p0)
cutsize=0
for i in range(len(p0)):
    if p0[i]==1:
        for j in vedge[i]:
            if p0[j]==0:
                cutsize+=1
print('Cross-Entropy method')
print('max cut includes',cutsize,'edges')

Cross-Entropy method
max cut includes 24 edges


In [16]:
######################### Q2(c) Monte Carlo Tree Search (MCTS) Method #########################
import random
import math
import hashlib
import logging
import numpy as np


vertice_num=20
edge_num=30
vedge=[[1, 4, 15], [0, 2, 14], [1, 3, 12], [2, 4, 7], [0, 3, 5], 
        [4, 6, 17], [5, 7, 8],[6, 10, 3], [6, 9, 18], [8, 10, 11], 
        [7, 9, 12], [9, 13, 19], [2, 10, 13], [12, 14, 11], [1, 13, 16], 
        [0, 16, 17], [14, 15, 19], [5, 15, 18], [8, 17, 19], [11, 16, 18]]

def expand(node):
    expanded=[c.state for c in node.children]
    nstate=node.state.next_state()
    while nstate in expanded:
        nstate=node.state.next_state()
    node.add_child(nstate)
    return node.children[-1]

def select(node):
    c=1/math.sqrt(2.0)
    R=[]
    select_node=[]
    for c in node.children:
        #UCB
        # val=self.vals[i]+self.c*(math.sqrt(2*math.log(t)/self.counts[i]))
        counts=c._n_visits
        exploit=c.reward/c._n_visits
        explore=math.sqrt(2.0*math.log(node._n_visits)/counts)    
        r=exploit+c*explore
        R.append(r)
    maxR=max(R)
    maxrs=np.where(R=maxR)
    idx=np.random.randint(0,len(maxrs))
    idx=R[0][idx]
    return node.children[idx]

def get_res(state):
    while state.terminal()==0:
        state=state.next_state()
    return state.reward()

def cal_cut(state):
    cutsize=0
    for i in range(len(state)):
        if state[i]==1:
            for j in vedge[i]:
                if state[j]==0:
                    cutsize+=1
    return cutsize

class State():
    def __init__(self, vertices=[0]*20):
        self.vertices=vertices
    def next_state(self):
        canflip=np.where(np.array(self.vertices)==0)
        idx=np.random.randint(0,len(canflip[0]))
        idx=canflip[0][idx]
        nstate=self.vertices[:]
        nstate[idx]=1
        return State(nstate)

    def terminal(self):
        if sum(self.vertices)==len(self.vertices):
            return 1
        return 0
    def reward(self):
        return cal_cut(self.vertices)
    
class TreeNode():
    def __init__(self, state, parent=None):
        self.children=[]
        self.parent=parent
        self._n_visits=0
        self.reward=0
        self.state=state
        # print('self.state',self.state)
    def add_child(self,child_state):
        child=TreeNode(child_state,self)
        self.children.append(child)
    def update(self,reward):
        self._n_visits+=1
        self.reward=cal_cut(cal_cut(self.state.vertices))
    def fully_expanded(self):
        if len(self.children)==vertice_num:
            return 1
        return 0

def MCTS(iters,root):
    for t in range(int(iters)):
        # if t%10==0:
            # print("simulation: %d"%t)
        # Selection, expansion
        while root.state.terminal()==0:
            if len(root.children)==0:
                return expand(root)
            elif random.uniform(0,1)<.5 or root.fully_expanded()==1:
                root=select(root)
            else:
                if root.fully_expanded()==0:    
                    return expand(root)
        pnode=root
        pstate=pnode.state
        # simu
        reward=get_res(pstate)
        # back prop
        while pnode!=None:
            pnode._n_visits+=1
            pnode.reward=cal_cut(pstate.vertices)
            pnode=pnode.parent
    return select(root)

if __name__=="__main__":
    iters=2000
    sims=300
    depth=20
    best_size=[]
    
    for t in range(iters):
        depth_cut=[]
        current_node=TreeNode(State())
        for d in range(depth):
            current_node=MCTS(sims/(d+1),current_node)
            best_child=current_node.state
            cur_size=cal_cut(best_child.vertices)
            depth_cut.append(cur_size)
        best_size.append(max(depth_cut))
    print('Monte Carlo Tree Search (MCTS) Method')
    print('max cut includes',max(best_size),'edges')

Monte Carlo Tree Search (MCTS) Method
max cut includes 24 edges


In [1]:
############################# Q4(a) Gomoku-forbidden_move #############################
import numpy as np

# check 1*6 quare for more than 5 in a row
def check_long(arr):
    blank=np.where(arr==0)
    black=np.where(arr==1)
    if len(blank[0])==1 and len(black[0])==5:
        return 1
    return 0

# check 1*5 square, one blank and three black apart from current location--four in a row
def check_4(arr):
    blank=np.where(arr==0)
    black=np.where(arr==1)
    if len(black[0])==4:
        return 2
    if len(blank[0])==2 and len(black[0])==3:
        return 1
    return 0

# check 1*4 square, one blank and two black apart from current location--three in a row
# Also must has two blank location on two sides.
def check_3_direct1(arr,i,j,whole_board,width):
    # any side not blank -> not 3
    if j==0 or j==width-4:
        return 0
    if (j>0 and whole_board[i][j-1]!=0) or (j+4<width and whole_board[i][j+4]!=0):
        return 0
    blank=np.where(arr==0)
    black=np.where(arr==1)
    if len(blank[0])==2 and len(black[0])==2:
        return 1
    return 0
def check_3_direct2(arr,i,j,whole_board,width):
    # any side not blank -> not 3
    if i==0 or i==width-4:
        return 0
    if (i>0 and whole_board[i-1][j]!=0) or (i+4<width and whole_board[i+4][j]!=0):
        return 0
    blank=np.where(arr==0)
    black=np.where(arr==1)
    if len(blank[0])==2 and len(black[0])==2:
        return 1
    return 0

def check_3_direct3(arr,i,j,whole_board,width):
    # any side not blank -> not 3
    if i==width-1 or j==0 or j==width-4 or i-3==0 or whole_board[i+1][j-1]!=0 or whole_board[i-4][j+4]!=0:
        # print('here')
        return 0
    blank=np.where(arr==0)
    black=np.where(arr==1)
    if len(blank[0])==2 and len(black[0])==2:
        return 1
    return 0

def check_3_direct4(arr,i,j,whole_board,width):
    # any side not blank -> not 3
    if i==0 or j==0 or j==width-4 or i==width-4 or whole_board[i-1][j-1]!=0 or whole_board[i+4][j+4]!=0:
        # print('here')
        return 0
    blank=np.where(arr==0)
    black=np.where(arr==1)
    if len(blank[0])==2 and len(black[0])==2:
        return 1
    return 0

# return 1 if x,y is forbiddened position
def forbidden_move(curPlayer,noncurPlayer,x,y,width):
    # if x==3 and y==3:
        # print(curPlayer,noncurPlayer)
    whole_board=np.copy(curPlayer)
    for i in range(width):
        noncur=np.where(np.array(noncurPlayer[i])==1)
        for j in noncur[0]:
            whole_board[i][j]=-1
    # for i in range(width):
    #     print(whole_board[i])
    cnt3=0
    cnt4=0
    cnt_long=0

    #### Direction1 vertical --- #######
    # 44: xooo?   xoo?o   _oo?o
    leftmost=max(y-4,0)
    rightmost=min(y+1,width-4)
    for j in range(leftmost,rightmost):
        arr=whole_board[x][j:j+5]
        flag4=check_4(np.array(arr))
        if flag4==2 and (j>0 and whole_board[x][j-1]!=1) and (j+5<width and whole_board[x][j+5]!=1):
            return 0 # win
        if flag4==1:
            cnt4+=1
            break
    # 33:
    leftmost=max(y-3,0)
    rightmost=min(y+1,width-3)
    for j in range(leftmost,rightmost):
        # length 4 window
        arr=whole_board[x][j:j+4]
        if check_3_direct1(np.array(arr),x,j,whole_board,width)==1:
            cnt3+=1
            break
    # long
    leftmost=max(y-5,0)
    rightmost=min(y+1,width-5)
    for j in range(leftmost,rightmost):
        # length 6 window
        arr=whole_board[x][j:j+6]
        if check_long(np.array(arr))==1:
            cnt_long+=1
            break

    #### Direction2 horizontal | #######
    # 44:
    leftmost=max(x-4,0)
    rightmost=min(x+1,width-4)
    for j in range(leftmost,rightmost):
        # length 5 window
        arr=[]
        for k in range(j,j+5):
            arr.append(whole_board[k][y])
        # print(arr)
        flag4=check_4(np.array(arr))
        if flag4==2 and (j>0 and whole_board[j-1][y]!=1) and (j+5<width and whole_board[j+5][y]!=1):
            # print(arr)
            # print(np.where(np.array(arr)==1))
            # print('2aa')
            return 0 # win
        if flag4==1:
            cnt4+=1
            break
    # 33:
    leftmost=max(x-3,0)
    rightmost=min(x+1,width-3)
    for j in range(leftmost,rightmost):
        # length 4 window
        arr=[]
        for k in range(j,j+4):
            arr.append(whole_board[k][y])
        if check_3_direct2(np.array(arr),j,y,whole_board,width)==1:
            cnt3+=1
            break

    # long
    leftmost=max(x-5,0)
    rightmost=min(x+1,width-5)
    for j in range(leftmost,rightmost):
        # length 6 window
        arr=[]
        for k in range(j,j+6):
            arr.append(whole_board[k][y])
        if check_long(np.array(arr))==1:
            cnt_long+=1
            break

    #### Direction3 upslope / #######
    # 44:
    if x+y>=4 and x+y<=10:
        tmp=min(7-x,y)
        # print(tmp)
        leftmost=min(tmp,4)
        tmp=min(x,7-y)
        rightmost=max(-4,-tmp)+3
        for j in range(leftmost,rightmost,-1):
            # length 5 window
            arr=[]
            for k in range(j,j-5,-1):
                arr.append(whole_board[x+k][y-k])
            # print(arr)
            flag4=check_4(np.array(arr))
            if flag4==2:
                if (x+j==width-1 or y-j==0 or (x+j+1<width and y-j>0 and whole_board[x+j+1][y-j-1]!=1)) and \
                   (y-j==width-5 or x+j-4==0 or (x+j-5>=0 and y-j+5<width and whole_board[x+j-5][y-j+5]!=1)):
                # print('3aa')
                    return 0 # win
            if flag4==1:
                cnt4+=1
                break
    # 33:
    if x+y>=3 and x+y<=11:
        tmp=min(7-x,y)
        leftmost=min(tmp,3)
        tmp=min(x,7-y)
        rightmost=max(-3,-tmp)+2
        for j in range(leftmost,rightmost,-1):
            # length 4 window
            arr=[]
            for k in range(j,j-4,-1):
                arr.append(whole_board[x+k][y-k])
            if check_3_direct3(np.array(arr),x+j,y-j,whole_board,width)==1:
                cnt3+=1
                break
    #long
    if x+y>=5 and x+y<=9:
        tmp=min(7-x,y)
        leftmost=min(tmp,5)
        tmp=min(x,7-y)
        rightmost=max(-5,-tmp)+4
        for j in range(leftmost,rightmost,-1):
            # length 6 window
            arr=[]
            for k in range(j,j-6,-1):
                arr.append(whole_board[x+k][y-k])
            if check_long(np.array(arr))==1:
                cnt_long+=1
                break
    
    #### Direction4 downslope \ #######
    if x-y<=3 and x-y>=-3:
        tmp=min(x,y)
        # print(tmp)
        leftmost=min(tmp,4)
        leftmost=-leftmost
        tmp=width-max(x,y)-1
        rightmost=min(tmp,4)-3
        for j in range(leftmost,rightmost):
            # length 5 window
            arr=[]
            for k in range(j,j+5):
                arr.append(whole_board[x+k][y+k])
            flag4=check_4(np.array(arr))
            if flag4==2:
                if(x+j==0 or y+j==0 or (x+j>0 and y+j>0 and whole_board[x+j-1][y+j-1]!=1)) and \
                   (x+j+4==width-1 or x+j+4==width-1 or (x+j+5<width and y+j+5<width and whole_board[x+j+5][y+j+5]!=1)):
                   
                   
                    return 0 # win
            if flag4==1:
                cnt4+=1
                break

    # 33:
    if x-y<=4 and x-y>=-4:
        tmp=min(x,y)
        # print(tmp)
        leftmost=min(tmp,3)
        leftmost=-leftmost
        tmp=width-max(x,y)-1
        rightmost=min(tmp,3)-2
        for j in range(leftmost,rightmost):
            # length 4 window
            arr=[]
            for k in range(j,j+4):
                arr.append(whole_board[x+k][y+k])
            if check_3_direct4(np.array(arr),x+j,y+j,whole_board,width)==1:
                cnt3+=1
                break

    #long
    if x-y<=2 and x-y>=-2:
        tmp=min(x,y)
        leftmost=min(tmp,5)
        leftmost=-leftmost
        tmp=width-max(x,y)-1
        rightmost=min(tmp,5)-4
        for j in range(leftmost,rightmost):
            # length 6 window
            arr=[]
            for k in range(j,j+6):
                arr.append(whole_board[x+k][y+k])
            if check_long(np.array(arr))==1:
                cnt_long+=1
                break
    
    # print('cnt4',cnt4)
    # print('cnt3',cnt3)
    # print('cnt_long',cnt_long)
    if cnt4>=2 or cnt3>=2 or cnt_long>=1:
        # print('FORBIDDEN')
        return 1
    return 0

#Debug
# flag=forbidden_move(curPlayer,noncurPlayer,3,3,8)
# flag=forbidden_move(curPlayer,noncurPlayer,4,4,8)
# flag=forbidden_move(curPlayer,noncurPlayer,2,1,8)
# flag=forbidden_move(curPlayer,noncurPlayer,3,3,8) #33

# if flag==1:
#     print('is forbidden')
# else:
#     print('not f')

# Qplayer
def cal_features(player,board,sensible_moves,thisplayerState,otherplayerState):
    # print(res.contblack2)
    # print(res.contblack3)
    for move in range(board.width * board.height):
        if move in sensible_moves:      
            for d in [1,7,8,9]:
                black = 0
                cnt = 1
                while ((move + d*cnt) in thisplayerState):
                    black += 1
                    cnt += 1
                cnt = 1
                while ((move - d*cnt) in thisplayerState):
                    black += 1
                    cnt += 1
                if black==1:
                    player.contblack2[move]=1
                elif black==2:
                    player.contblack3[move]=1
                elif black==3:
                    player.contblack4[move]=1
                elif black==4:
                    player.contblack5[move]=1
    for move in range(board.width * board.height):
        if move in sensible_moves:      
            for d in [1,7,8,9]:
                white = 0
                cnt = 1
                while ((move + d*cnt) in otherplayerState):
                    white += 1
                    cnt += 1
                cnt = 1
                while ((move - d*cnt) in otherplayerState):
                    white += 1
                    cnt += 1
                if white==2:
                    player.contwhite3[move]=1
                elif white==3:
                    player.contwhite4[move]=1
                elif white==4:
                    player.contwhite5[move]=1
    # return player


In [3]:
############################# Q4(b) Pure RL player #############################

import numpy as np
from Forbidden_Moves import cal_features
import copy
import math
EPS=0.5
def flip_coin(board):
    explore=EPS/math.log(len(board.states)+0.01)
    gen=np.random.random()
    if gen<explore:
        return 1
    return 0

class QPlayer:
    # method: qlearn
    def __init__(self,board,alpha=0.1,gamma=0.9):
        self.alpha = alpha
        self.gamma = gamma # future reward discount factor
        self.Q=np.zeros(board.width * board.height)
        #执黑
        #feature1-7
        self.contblack2=np.zeros(board.width * board.height)
        self.contblack3=np.zeros(board.width * board.height)
        self.contblack4=np.zeros(board.width * board.height)
        self.contblack5=np.zeros(board.width * board.height)
        self.contwhite3=np.zeros(board.width * board.height)
        self.contwhite4=np.zeros(board.width * board.height)
        self.contwhite5=np.zeros(board.width * board.height)
        self.weights=[5, 10, 20, 15, 30]
        #             b2  b3  b4  w3  w4
        # self.weights=[0.5, 1, 2, 5, 1.5, 3, 5]
        #             b2  b3  b4 b5  w3 w4 w5
        self.player=None
    def update_qval(self,board,sensible_moves):
        # print(board.states)
        thisplayerState=[]
        otherplayerState=[]
        for k in board.states:
            if board.states[k]==self.player:
                thisplayerState.append(k)
            else:
                otherplayerState.append(k)
        cal_features(self,board,sensible_moves,thisplayerState,otherplayerState)
        # print(len(np.where(self.contwhite3)[0]))
        for move in range(board.width * board.height):
            if move in sensible_moves:
                if self.contblack5[move]==1:
                    self.Q[move]=500
                elif self.contwhite5[move]==1:
                    self.Q[move]=500
                else:
                    features=self.weights[0] * self.contblack2[move]+\
                            self.weights[1] * self.contblack3[move]+\
                            self.weights[2] * self.contblack4[move]+\
                            self.weights[3] * self.contwhite3[move]+\
                            self.weights[4] * self.contwhite4[move]
                    self.Q[move]=features
            else:
                self.Q[move]=-100000
            if move in board.save_forbidden:
                self.Q[move]=-100000
    def update_weights(self,move,prevQ):
        maxQ=max(self.Q)
        upd = 0.01 + self.gamma*maxQ - prevQ
        coef=self.alpha*upd
        self.weights[0]+=coef*self.contblack2[move]
        self.weights[1]+=coef*self.contblack3[move]
        self.weights[2]+=coef*self.contblack4[move]
        self.weights[3]+=coef*self.contwhite3[move]
        self.weights[4]+=coef*self.contwhite4[move]
        
    def set_player_ind(self, p):
        self.player = p

    def get_action(self,board):
        # print('get Q action')
        sensible_moves=np.copy(board.availables)
        for m in sensible_moves:
            if m in board.save_forbidden:
                np.delete(sensible_moves,np.where(sensible_moves==m)[0])
        iters=250
        for t in range(iters):
            self.update_qval(board,sensible_moves)
            if flip_coin(board)==1:
                move=np.random.choice(sensible_moves)
            else:
                maxQ=max(self.Q)
                best_choices=np.where(self.Q==maxQ)[0]
                
                for m in best_choices:
                    if m in board.save_forbidden:
                        # np.delete(best_choices,m)
                        np.delete(best_choices,np.where(best_choices==m)[0])
                move=np.random.choice(best_choices)
            tmpBoard=copy.deepcopy(board)
            tmpBoard.do_move(move)
            otherm=np.copy(board.availables)
            for m in otherm:
                if m in board.save_forbidden:
                    # otherm.remove(m)
                    np.delete(otherm,np.where(otherm==m)[0])

            randmove=np.random.choice(otherm)
            prevQ=self.Q[move]
            self.update_qval(board,sensible_moves)
            self.update_weights(move,prevQ)

        return move
    def __str__(self):
        return "Pure RL Player"